# Import Libraries and Datasets

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from src.constants import BATCH_SIZE, IMAGE_SIZE, CHANNELS
from src.preprocessing_functions import create_generator
from src.models import build_cnn_model

In [2]:
# Load paths in a dict for train, test, valid folders with wildfire and nowildfire images 

images_path = '/Users/silvanoquarto/Desktop/PROJECTS/Data_Wildfire_Project'
splits = ['train', 'test', 'valid']
labels = ['wildfire', 'nowildfire']

image_paths = {f"{split}_{label}": [] for split in splits for label in labels}

for split in splits:
    for label in labels:
        path = os.path.join(images_path, split, label)
        if os.path.exists(path):  
            image_files = os.listdir(path)
            full_paths = [os.path.join(path, img) for img in image_files]
            image_paths[f"{split}_{label}"].extend(full_paths)
            print(f"- {split.capitalize()} set for {label} uploaded correctly!!")
        else:
            print(f"Warning: {path} does not exist :(")

- Train set for wildfire uploaded correctly!!
- Train set for nowildfire uploaded correctly!!
- Test set for wildfire uploaded correctly!!
- Test set for nowildfire uploaded correctly!!
- Valid set for wildfire uploaded correctly!!
- Valid set for nowildfire uploaded correctly!!


# IMAGE PREPROCESSING

In [ ]:
# Data augumentation and normalization
 
print('Data augumentation and normalization for training...')
train_generator = create_generator(images_path, 'train')

print('Data augumentation and normalization for validation...')
validation_generator = create_generator(images_path, 'valid')

print('Data augumentation and normalization for test...')
test_generator = create_generator(images_path, 'test')

# Build and Train CNN model from scratch

## Build model

In [ ]:
model = build_cnn_model(input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS), num_output=1, learning_rate=0.001, dropout_rate=0.5)

model.summary()

## Train model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

# Model evaluation

In [ ]:
# Evaluate the model on the test set

test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

# Save model

In [ ]:
# Save the model

model.save('wildfire_detection_cnn.h5')